In [ ]:
print("starting 8_blur_activations_and_save_maximally_activating_blur_img")

# Investigate Occlusion Stimuli

## Imports

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# general imports
import numpy as np
import math
import random
import pandas as pd
import tensorflow as tf
import os
import cv2
import time
import copy
import argparse
from pathlib import Path
from tqdm import tqdm

In [ ]:
# lucid imports
import lucid.modelzoo.vision_models as models
from render import import_model

In [ ]:
# custom imports
import occlusion_utils as ut

## Load model

In [ ]:
# import InceptionV1 from the Lucid modelzoo
model = models.InceptionV1()
model.load_graphdef()

## Parameters

In [ ]:
batch_size_forward_pass = 512

In [ ]:
# setting seeds
tf.set_random_seed(1234)
random.seed(0)
np.random.seed(0)

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("-s", "--stimuli-dir", required=True, help="Path to save stimuli to.")
parser.add_argument("-t", "--trial-type", required=True, help="instruction_practice_catch or sampled_trials.")
args = parser.parse_args()
print(args)

In [ ]:
stimuli_dir = args.stimuli_dir
trial_type = args.trial_type

In [ ]:
# TODO: test with this cell, then change the upper two cells back
# stimuli_dir = "$DATAPATH/stimuli/stimuli_pure_conditions"
# trial_type = "sampled_trials"

## Load experiment specification

In [ ]:
# read in unit specifications from csv into pandas dataframe
path_to_csv_file = os.path.join(stimuli_dir, f"layer_folder_mapping_{trial_type}.csv")
unit_specs_df = pd.read_csv(path_to_csv_file, header=1)

In [ ]:
unit_specs_df

In [ ]:
# loop over all layers
for _, row in tqdm(unit_specs_df.iterrows(), unit="row", total=len(unit_specs_df), position=0):
    # load unit specification
    layer_number = row["layer_number"]
    kernel_size_number = row["kernel_size_number"]
    channel_number = row["channel_number"]
    feature_map_number = row["feature_map_number"]
    layer_name = row["layer_name"]
    pre_post_relu = row["pre_post_relu"]

    start = time.time()

    # dataloader
    data_dir = os.path.join(
        stimuli_dir,
        ut.objective,
        trial_type,
        f"layer_{layer_number}",
        f"kernel_size_{kernel_size_number}",
        f"channel_{channel_number}",
        "natural_images",
    )

    data_loader = ut.get_data_loader(data_dir, do_resize_and_center_crop=False)

    # loop over all occlusion sizes
    for heatmap_size_i, occlusion_size_i, percentage_side_length_i in zip(ut.heatmap_sizes_list,
                                                                          ut.occlusion_sizes_list,
                                                                          ut.percentage_side_length_list):

        if percentage_side_length_i != 40:
            continue
        print("Patch Size:", percentage_side_length_i)

        list_of_positions = ut.get_list_of_occlusion_positions(heatmap_size_i, occlusion_size_i)

        if 'session' in locals() and session is not None:
            print('Close interactive session')
            session.close()

        with tf.Graph().as_default() as graph, tf.Session() as sess:

            image = tf.placeholder(tf.float32, shape=(batch_size_forward_pass, 224, 224, 3))

            model_instance = import_model(model, image)
            tf_activations_list, unique_layer_str_list = ut.get_tf_activations_list(model_instance, layer_name, pre_post_relu)

            # loop over all images in all batches
            for images, targets, paths in tqdm(data_loader, leave=False, position=1, unit="image", total=len(data_loader.dataset)):
                # only evaluate reference images, no query images
                if "reference_max" not in paths[0]:
                    continue

                batch_name = Path(paths[0]).parts[-2]

                destination_dir = os.path.join(
                    stimuli_dir,
                    ut.objective,
                    trial_type,
                    f"layer_{layer_number}",
                    f"kernel_size_{kernel_size_number}",
                    f"channel_{channel_number}",
                    "natural_blur_images",
                    f"{batch_name}"
                )
                destination_dir = os.path.join(
                    destination_dir,
                    f"{percentage_side_length_i}_percent_side_length"
                )

                os.makedirs(destination_dir, exist_ok=True)
                final_image_path = os.path.join(destination_dir, f"blurred_{os.path.basename(paths[0]).split('.png')[0]}.png")

                if os.path.exists(final_image_path):
                    continue

                # iterate over all occlusion positions for the current image
                list_of_activations_for_occlusions = []
                n_iterations = math.ceil(len(list_of_positions) / batch_size_forward_pass)
                for iteration_i in tqdm(list(range(n_iterations)), unit="position", leave=False, position=2):
                    cur_list_of_positions = list_of_positions[iteration_i*batch_size_forward_pass:
                                                              iteration_i*batch_size_forward_pass+batch_size_forward_pass]

                    # forward pass
                    images_np_transformed = images.numpy().transpose(0,2,3,1) # (1, 224, 224, 3)
                    images_np_transformed_copied = copy.deepcopy(images_np_transformed)
                    images_np_transformed_ready_for_occlusion = np.tile(images_np_transformed_copied, (batch_size_forward_pass, 1, 1, 1)) # (batch_size_forward_pass, 224, 224, 3)

                    # loop through occlusion positions
                    for idx, (x_start, x_end, y_start, y_end) in enumerate(cur_list_of_positions):
                        patch_to_keep = images_np_transformed_copied[0, x_start:x_end, y_start:y_end, :]
                        images_np_transformed_ready_for_occlusion[idx, :, :, :] = cv2.GaussianBlur(images_np_transformed_ready_for_occlusion[idx], (21,21), 0)
                        images_np_transformed_ready_for_occlusion[idx, x_start:x_end, y_start:y_end, :] = patch_to_keep

                    activations_list = sess.run(tf_activations_list, {image: images_np_transformed_ready_for_occlusion})

                    # unpack single list item
                    activations_np = activations_list[0]
                    unit_activations = ut.get_activation_according_to_objective(ut.objective, activations_np, feature_map_number)
                    list_of_activations_for_occlusions.append(list(unit_activations))

                # after having calculated all occlusions: save activations
                final_list_of_activations_for_occlusions = [item for sublist in list_of_activations_for_occlusions for item in sublist]

                filename = os.path.join(destination_dir, f"activations_for_occlusions_of_{percentage_side_length_i}_percent_{os.path.basename(paths[0]).split('.png')[0]}.npy")
                activations_data_one_occlusion_size = final_list_of_activations_for_occlusions[:len(list_of_positions)+1]
                np.save(filename, activations_data_one_occlusion_size)

                # save image of maximal activation with blur
                extreme_idx = np.argmax(activations_data_one_occlusion_size[:-1])
                x_start, x_end, y_start, y_end = list_of_positions[extreme_idx]

                image_to_be_saved = copy.deepcopy(images_np_transformed.squeeze())
                image_to_be_saved_for_patch = copy.deepcopy(image_to_be_saved)
                patch_to_keep = image_to_be_saved_for_patch[x_start:x_end, y_start:y_end, :]
                image_to_be_saved[:, :, :] = cv2.GaussianBlur(image_to_be_saved, (21,21), 0)
                image_to_be_saved[x_start:x_end, y_start:y_end, :] = patch_to_keep

                cv2.imwrite(final_image_path, cv2.cvtColor(image_to_be_saved*255, cv2.COLOR_RGB2BGR))

    print("Completely done!")

In [ ]:
print("done with 8_blur_activations_and_save_maximally_activating_blur_img")